<a href="https://colab.research.google.com/github/ishankaul1/cs5262/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Intro**

For this assignment, I will be undoing and experimenting with my feature engineering for the 'timestamp' column in my dataset. 
In the previous assignments, I transformed the timestamp column into 'seasons' and 'during_restrictions' (meaning whether or not the data was collected during covid restrictions), and then dropped the original timestamp column. However, I never actually performed testing on whether this feature engineering plan affected the model. For this assignment, I will run the same model over several different configurations of this feature engineering task to determine whether or not it actually improves the model.

In addition, since I'm also curious to know whether my data is a better predictor of stress than the combined version of stress and shouting, I'll be dropping the shout column completely and only attempting to predict whether stress levels were greater than a '2' on a scale of 0 to 5.

My feature engineering plan for the experiment is:

1) Produce 4 different 'versions' of the same dataset. All will have the original Timestamp, Daily Shouting, and Wellbeing Score columns dropped, with the rest of the data intact and transformed the same way I did it in Assignments 1-5.
  Version 1: Both during_restrictions and season are in the dataset.
  Version 2: Only seasons is contained in the dataset.
  Version 3: Only during_restrictions is contained in the dataset.
  Version 4 (Control): Neither column in the dataset.
2) Create 4 different pipelines, each fed by the different datasets I created in step 1). The rest of the pipeline will be the exact same as the original Random Forest pipeline I ran in Assignment 5, except with a smaller range of hyperparameters centered around where my model seemed to perform best.
3) Compare the results of the best estimator for the 4 different models to determine whether either of these columns actually have any predictive power towards daily stress.